In [26]:
import numpy as np
import pandas as pd
from pathlib import Path

In [27]:
PATH_FILE_DATA: Path = Path("exports/zorgkaart_details.xlsx")
PATH_FILE_GEM: Path = Path("../20250625 - GemeenteNamen.xlsx")
PATH_FILE_OUTPUT: Path = Path("../20250625 - Data Zorgkaart.xlsx")

In [28]:
df = pd.read_excel(PATH_FILE_DATA)

In [29]:
gem_df = pd.read_excel(PATH_FILE_GEM)

In [30]:
gem_df.tail(5)

,Id,Measure,Woonplaatsen,Value,StringValue
2523,11621,GM000C,WP1262,’s-Heer Abtskerke,Borsele
2524,12216,GM000C,WP1399,Achterberg,Rhenen
2525,17172,GM000C,WP3385,Buren,Buren
2526,3137,GM000C,WP2729,Dronryp,Waadhoeke
2527,9213,GM000C,WP3402,Metslawier,Noardeast-Fryslân


# Plaatsnaam Bergen veranderen

In [31]:
df.loc[df['plaats'] == 'Bergen', 'plaats'] = df.loc[df['plaats'] == 'Bergen'].apply(
    lambda row: 'Bergen (NH)' if str(row['postcode']).startswith('18') else 'Bergen L',
    axis=1
)

# Toevoegen GemeenteNamen

In [32]:
df = df.merge(
    gem_df[['Value', 'StringValue']],
    left_on='plaats',
    right_on='Value',
    how='left'
)

In [33]:
df.rename(columns={'StringValue': 'GemeenteNaam'}, inplace=True)

In [34]:
df.drop(columns=['Value'], inplace=True)

In [35]:
df[df['plaats'] == 'Achterberg']

,organisatietype,naam,url,straat,huisnummer,toevoeging,postcode,plaats,scraped_at,GemeenteNaam
1047,Verpleeghuis en verzorgingshuis,"QuaRijn, De Linde",https://www.zorgkaartnederland.nl/zorginstelli...,De Grote Akker,22.0,NaN,3911XS,Achterberg,2025-05-28,Rhenen
26727,(Medisch) Pedicurepraktijk,Cura Piedi,https://www.zorgkaartnederland.nl/zorginstelli...,De Horst,61.0,NaN,3911SX,Achterberg,2025-06-10,Rhenen


In [36]:
df[df['GemeenteNaam'].isna()]

,organisatietype,naam,url,straat,huisnummer,toevoeging,postcode,plaats,scraped_at,GemeenteNaam
41824,Psychologiepraktijk en psychotherapiepraktijk,Praktijk Christel Grieten,https://www.zorgkaartnederland.nl/zorginstelli...,Mosselgoren,149.0,NaN,2440,Geel (BE),2025-06-10,NaN


# Dataframe back to Excel

In [37]:
df.to_excel(PATH_FILE_OUTPUT, index=False)